### Librerias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import holidays
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from functools import partial# import ace_tools as tools

### Directorios


In [2]:
DATOS_DIR = '../data/'
SALIDAS_DIR = '../salidas/'

Lectura de datos

In [3]:
FILE_NAME_SELL_IN='sell_in_fixed36.parquet'
FILE_NAME_PRODUCTOS='tb_productos.txt'

df = pd.read_parquet(DATOS_DIR + FILE_NAME_SELL_IN)
df_productos = pd.read_csv(DATOS_DIR + FILE_NAME_PRODUCTOS , sep='\t')
df = pd.merge(df, df_productos, on='product_id', how='left')
# Ordenar el dataset por las columnas especificadas
df = df.sort_values(by=["customer_id", "product_id", "periodo_dt"])

In [4]:
df.info()
df.head(40)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16422912 entries, 0 to 16422911
Data columns (total 14 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   customer_id            int32         
 1   product_id             int32         
 2   periodo_dt             datetime64[ns]
 3   plan_precios_cuidados  int8          
 4   cust_request_qty       float64       
 5   cust_request_tn        float32       
 6   tn                     float32       
 7   is_synth               object        
 8   is_synth_36            bool          
 9   cat1                   object        
 10  cat2                   object        
 11  cat3                   object        
 12  brand                  object        
 13  sku_size               int64         
dtypes: bool(1), datetime64[ns](1), float32(2), float64(1), int32(2), int64(1), int8(1), object(5)
memory usage: 1.3+ GB


,customer_id,product_id,periodo_dt,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,is_synth,is_synth_36,cat1,cat2,cat3,brand,sku_size
0,10001,20001,2017-01-01,0,11.0,99.438606,99.438606,False,False,HC,ROPA LAVADO,Liquido,ARIEL,3000
1,10001,20001,2017-02-01,0,23.0,198.843643,198.843643,False,False,HC,ROPA LAVADO,Liquido,ARIEL,3000
2,10001,20001,2017-03-01,0,33.0,92.465370,92.465370,False,False,HC,ROPA LAVADO,Liquido,ARIEL,3000
3,10001,20001,2017-04-01,0,8.0,13.297280,13.297280,False,False,HC,ROPA LAVADO,Liquido,ARIEL,3000
4,10001,20001,2017-05-01,0,15.0,101.207108,101.005630,False,False,HC,ROPA LAVADO,Liquido,ARIEL,3000
5,10001,20001,2017-06-01,0,25.0,128.047913,128.047913,False,False,HC,ROPA LAVADO,Liquido,ARIEL,3000
6,10001,20001,2017-07-01,0,18.0,101.207108,101.207108,False,False,HC,ROPA LAVADO,Liquido,ARIEL,3000
7,10001,20001,2017-08-01,0,17.0,43.339298,43.339298,False,False,HC,ROPA LAVADO,Liquido,ARIEL,3000
8,10001,20001,2017-09-01,0,36.0,289.350250,289.350250,False,False,HC,ROPA LAVADO,Liquido,ARIEL,3000
9,10001,20001,2017-10-01,0,22.0,222.113892,222.113892,False,False,HC,ROPA LAVADO,Liquido,ARIEL,3000


### Agrego periodo de 0 a 35

In [5]:
periodo_mapping = {
    pd.to_datetime("2017-01-01"): 0, pd.to_datetime("2017-02-01"): 1, pd.to_datetime("2017-03-01"): 2,
    pd.to_datetime("2017-04-01"): 3, pd.to_datetime("2017-05-01"): 4, pd.to_datetime("2017-06-01"): 5,
    pd.to_datetime("2017-07-01"): 6, pd.to_datetime("2017-08-01"): 7, pd.to_datetime("2017-09-01"): 8,
    pd.to_datetime("2017-10-01"): 9, pd.to_datetime("2017-11-01"): 10, pd.to_datetime("2017-12-01"): 11,
    pd.to_datetime("2018-01-01"): 12, pd.to_datetime("2018-02-01"): 13, pd.to_datetime("2018-03-01"): 14,
    pd.to_datetime("2018-04-01"): 15, pd.to_datetime("2018-05-01"): 16, pd.to_datetime("2018-06-01"): 17,
    pd.to_datetime("2018-07-01"): 18, pd.to_datetime("2018-08-01"): 19, pd.to_datetime("2018-09-01"): 20,
    pd.to_datetime("2018-10-01"): 21, pd.to_datetime("2018-11-01"): 22, pd.to_datetime("2018-12-01"): 23,
    pd.to_datetime("2019-01-01"): 24, pd.to_datetime("2019-02-01"): 25, pd.to_datetime("2019-03-01"): 26,
    pd.to_datetime("2019-04-01"): 27, pd.to_datetime("2019-05-01"): 28, pd.to_datetime("2019-06-01"): 29,
    pd.to_datetime("2019-07-01"): 30, pd.to_datetime("2019-08-01"): 31, pd.to_datetime("2019-09-01"): 32,
    pd.to_datetime("2019-10-01"): 33, pd.to_datetime("2019-11-01"): 34, pd.to_datetime("2019-12-01"): 35
}

# Agregar la columna periodo_indice al DataFrame usando el mapeo
df['periodo_indice'] = df['periodo_dt'].map(periodo_mapping)

### Agrego nuevas caracteristicas basadas en calendario

In [6]:
df["year"]        = df["periodo_dt"].dt.year.astype("int16")
df["month"]       = df["periodo_dt"].dt.month.astype("int8")
df["quarter"]     = df["periodo_dt"].dt.quarter.astype("int8")
df["month_idx"]   = (df["year"] * 12 + df["month"]).astype("int32")  # continuous index
df["is_fy_start"] = df["month"].eq(1).astype("int8")
df["is_fy_end"]   = df["month"].eq(12).astype("int8")

In [7]:

# -----------------------------------------------------------
# 1. Seasons  (southern hemisphere)
#    Summer = Dec–Feb, Autumn = Mar–May, Winter = Jun–Aug, Spring = Sep–Nov
# -----------------------------------------------------------
season_map = {
    12: "summer", 1: "summer", 2: "summer",
    3:  "autumn", 4: "autumn", 5: "autumn",
    6:  "winter", 7: "winter", 8: "winter",
    9:  "spring", 10: "spring", 11: "spring",
}

df["season"] = (
    df["periodo_dt"].dt.month.map(season_map)
      .astype("category")              # LightGBM friendly
)

In [ ]:
# VER DE AGREGAR CARACTERISTICAS QUE INDIQUEN VACACIONES Y FERIADOS, NAVIDAD, ETC.
# ar_holidays = holidays.AR()
# for ptr in holidays.AR(years = 2018).items():
#     p, int(ptr)

### AGREGO MESES EXTRAÑOS EN EL DATASET

Mes atípico:
- Junio 2019
- Agosto 2019

In [8]:
df["mes_anormal_1"] = (
    df["periodo_dt"].dt.to_period("M") == pd.Period("2019-08", freq="M")
).astype("int8")          # 1 para agosto-2019, 0 para el resto

df["mes_anormal_2"] = (
    df["periodo_dt"].dt.to_period("M") == pd.Period("2019-06", freq="M")
).astype("int8")          # 1 para agosto-2019, 0 para el resto

### Creo clase y lags

In [9]:
df = df.sort_values(["product_id","customer_id", "periodo_dt"])

# tn adelantado 2 periodos (h=2)
df["clase"] = (
    df.groupby(["product_id","customer_id"])["tn"]
      .shift(-2)          # -2 = lead de 2 meses
      .astype("float32")  # mismo tipo que tn para ahorrar RAM
)

# Generar columnas tn-1, tn-2, ..., tn-11 (12 meses de adelanto)
for i in range(1, 12):
    df[f"tn-{i}"] = (
        df.groupby(["product_id", "customer_id"])["tn"]
          .shift(+i)          # -i = lead de i meses
          .astype("float32")  # mismo tipo que tn para ahorrar RAM
    )


### Rolling: media y desviación en ventana de 3,6 y 12 meses
shift(1) para evitar "ver el futuro"

In [10]:
grp = df.groupby(["customer_id", "product_id"])
# ───────────────────────────────
# Rolling: media y desviación en ventana de 3,6 y 12 meses
# ───────────────────────────────
window_sizes = [3, 6, 12]

for w in window_sizes:
    # Media móvil
    df[f"tn_roll_mean_{w}"] = (
        grp["tn"]
          .rolling(window=w)
          .mean()
          .shift(1)                       # deja fuera el mes actual
          .reset_index(level=[0,1], drop=True)
          .astype("float32")
    )
    # Desviación estándar móvil
    df[f"tn_roll_std_{w}"] = (
        grp["tn"]
          .rolling(window=w)
          .std()
          .shift(1)
          .reset_index(level=[0,1], drop=True)
          .astype("float32")
    )

# ───────────────────────────────
# 3. Opcional: rellenar NaN iniciales con 0 o dejarlos
# ───────────────────────────────
# lag_cols = [f"tn_lag{l}" for l in lag_list]
roll_cols = [f"tn_roll_mean_{w}" for w in window_sizes] + \
            [f"tn_roll_std_{w}"  for w in window_sizes]

df[roll_cols] = df[roll_cols].fillna(0)

# Guardar si quieres
# df.to_parquet("data/sell_in_fixed36_feats.parquet", compression="zstd")


In [11]:
# borro columnas que no voy a usar
columnas_a_borrar = ["periodo_dt", "is_synth", "is_synth_36"]
df.drop(columns=columnas_a_borrar, inplace=True, errors='ignore')

### GUARDO UN PRIMER DATASET EN BASE A ESTE NOTEBOOK

In [13]:
FILE_NAME_SALIDA='sell_in_03-featureaddv1.parquet"'
df.to_parquet(SALIDAS_DIR + FILE_NAME_SALIDA , compression="zstd")